In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold as KF
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import warnings
warnings.simplefilter('ignore')


train = pd.read_csv(rf'train.csv')
test = pd.read_csv(rf'test.csv')

train_y = train["y"] 
train_x = train.drop(columns = ["y"])
test_x = test



# マッピング
marital_mapping = {'married': 1.0, 'single': 0.0, 'divorced': 0.0}
train_x["Marital"] = train_x["marital"].map(marital_mapping)
test_x["Marital"] = test_x["marital"].map(marital_mapping)

education_mapping = {'primary': 1, 'secondary': 2, 'tertiary': 3, 'unknown': 3}
train_x["Education"] = train_x["education"].map(education_mapping)
test_x["Education"] = test_x["education"].map(education_mapping)




# testデータの["job"]列に"unknown"があるので，最頻値"student"で置き換え
test_x.loc[test_x["job"] == "unknown"] = "student"



# student or unemployedで20～40歳の場合を1,他を0
train_x["job_sp"] = 0
test_x["job_sp"] = 0

train_x["job_sp"].loc[(train_x["job"] == "student") | 
            ((train_x["job"] == "unemployed") & (train_x["age"] >= 20) & (train_x["age"] <= 40))] = 1

test_x["job_sp"].loc[(test_x["job"] == "student") | 
            ((test_x["job"] == "unemployed") & (test_x["age"] >= 20) & (test_x["age"] <= 40))] = 1
        

    
# 総接触回数        
train_x["campaign_previous"] = train_x["campaign"] + train_x["previous"]
test_x["campaign_previous"] = test_x["campaign"] + test_x["previous"] 
        

    

# 10年代当たりの接触回数
age_ctgr_list = [0, 20, 30, 40, 50, 60, 100]
age_ctgr_name = ['1', '2', '3', '4', '5', '6']
 
train_x["Age"] = pd.cut(train_x["age"], bins=age_ctgr_list, labels=age_ctgr_name)
test_x["Age"] = pd.cut(test_x["age"], bins=age_ctgr_list, labels=age_ctgr_name)

train_x["Age"] = train_x["Age"].astype(int)
test_x["Age"] = test_x["Age"].astype(int)


train_x["p+c/A"] = (train_x["previous"] + train_x["campaign"]) / train_x["Age"]
test_x["p+c/A"] = (test_x["previous"] + test_x["campaign"]) / test_x["Age"]




# ラベルエンコーディング
le = LabelEncoder() 
train_x["marital"] = le.fit_transform(train_x["marital"])
test_x["marital"] = le.fit_transform(test_x["marital"])

train_x["education"] = le.fit_transform(train_x["education"])
test_x["education"] = le.fit_transform(test_x["education"])

train_x["contact"] = le.fit_transform(train_x["contact"])
test_x["contact"] = le.fit_transform(test_x["contact"])

train_x["poutcome"] = le.fit_transform(train_x["poutcome"])
test_x["poutcome"] = le.fit_transform(test_x["poutcome"])




# ワンホットエンコーディング
ohe_tr = pd.get_dummies(train_x[["job"]])
ohe_te = pd.get_dummies(test_x[["job"]])
ohe_tr["id"] = train_x["id"]
ohe_te["id"] = test_x["id"]

all_tr = pd.merge(train_x, ohe_tr, how = "left")
all_te = pd.merge(test_x, ohe_te, how = "left")


train_x = all_tr.drop(columns = ["job", "Marital", "Education"])
test_x = all_te.drop(columns = ["job", "Marital", "Education"])




# 日付数値化
month_mapping = {'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6, 'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, "nov": 11, 'dec': 12}
train_x["month"] = train_x["month"].map(month_mapping)
train_x["DAY"] = train_x["month"]*100 + train_x["day"]
test_x["month"] = test_x["month"].map(month_mapping)
test_x["DAY"] = test_x["month"]*100 + test_x["day"]




loan_mapping = {'no': 0, 'yes': 1}
train_x["loan"] = train_x["loan"].map(loan_mapping)
train_x["housing"] = train_x["housing"].map(loan_mapping)
train_x["balance"] = train_x["loan"] + train_x["housing"]
train_x["default"] = train_x["default"].map(loan_mapping)

test_x["loan"] = test_x["loan"].map(loan_mapping)
test_x["housing"] = test_x["housing"].map(loan_mapping)
test_x["balance"] = test_x["loan"] + test_x["housing"]
test_x["default"] = test_x["default"].map(loan_mapping)




# ageとdurationだけ標準化
stdsc = StandardScaler()

scaler = StandardScaler()
num_cols = ['age', 'duration']
scaler.fit(train_x[num_cols])

train_x[num_cols] = scaler.transform(train_x[num_cols])
test_x[num_cols] = scaler.transform(test_x[num_cols])
train_x = train_x.astype(float)
test_x = test_x.astype(float)
   
    
    
    
train_x = train_x.drop(columns = ["id", "pdays", "default", "job_self-employed"])
test_x = test_x.drop(columns = ["id", "pdays" ,"default", "job_self-employed"]) 




scores = []
splits = 5


# 学習データを5分割，うち１個をバリデーションデータにする
kf = KF(n_splits = splits, shuffle = True, random_state = 71)
tr_idx, va_idx = list(kf.split(train_x))[0]  # 5分割しているため，[0]から[4]まで指定できる
# tr_idxにはlist[1～4]が，va_idxにはlist[0]が代入される
for tr_idx, va_idx in kf.split(train_x):
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
    
    
    # xgbbostが扱える変数の形にする
    dtrain = xgb.DMatrix(tr_x, label = tr_y)
    dvalid = xgb.DMatrix(va_x, label = va_y)
    dtest = xgb.DMatrix(test_x)
    
    # ハイパーパラメータの設定
    params = {'objective': 'binary:logistic', 'silent': 1, 'random_state': 71, 'max_depth': 3, 'colsample_bytree': 0.8, 'min_child_weight': 15} 
    
    num_round = 200
    
    watchlist = [(dtrain, 'train'), (dvalid, 'evaluate')]
    Model1Xgb = xgb.train(params, dtrain, num_round, evals = watchlist, early_stopping_rounds = 25)
    
    va_pred = Model1Xgb.predict(dvalid)
    score = roc_auc_score(va_y, va_pred)
    scores.append(score)


print(np.mean(scores))





ans = Model1Xgb.predict(dtest)
# PassengerIdを取得
PassengerId = np.array(test["id"]).astype(int)
 
# my_prediction(予測データ）とPassengerIdをデータフレームへ落とし込む
my_solution = pd.DataFrame(ans, PassengerId)
 
# my_tree_one.csvとして書き出し
my_solution.to_csv(rf'answer.csv', header = False)

[0]	train-error:0.071956	evaluate-error:0.068635
Multiple eval metrics have been passed: 'evaluate-error' will be used for early stopping.

Will train until evaluate-error hasn't improved in 25 rounds.
[1]	train-error:0.073708	evaluate-error:0.070849
[2]	train-error:0.071956	evaluate-error:0.068635
[3]	train-error:0.071956	evaluate-error:0.068635
[4]	train-error:0.071771	evaluate-error:0.068819
[5]	train-error:0.071771	evaluate-error:0.069004
[6]	train-error:0.071863	evaluate-error:0.069373
[7]	train-error:0.072371	evaluate-error:0.069188
[8]	train-error:0.07274	evaluate-error:0.069557
[9]	train-error:0.072648	evaluate-error:0.069557
[10]	train-error:0.07214	evaluate-error:0.069373
[11]	train-error:0.072002	evaluate-error:0.069188
[12]	train-error:0.07131	evaluate-error:0.068635
[13]	train-error:0.071633	evaluate-error:0.068266
[14]	train-error:0.071264	evaluate-error:0.068081
[15]	train-error:0.071218	evaluate-error:0.068081
[16]	train-error:0.070987	evaluate-error:0.067897
[17]	train

[16]	train-error:0.069742	evaluate-error:0.071402
[17]	train-error:0.06928	evaluate-error:0.071218
[18]	train-error:0.069234	evaluate-error:0.071033
[19]	train-error:0.06905	evaluate-error:0.070849
[20]	train-error:0.068911	evaluate-error:0.070849
[21]	train-error:0.068496	evaluate-error:0.070295
[22]	train-error:0.068081	evaluate-error:0.07048
[23]	train-error:0.067897	evaluate-error:0.07048
[24]	train-error:0.067897	evaluate-error:0.070295
[25]	train-error:0.067758	evaluate-error:0.070295
[26]	train-error:0.067758	evaluate-error:0.07048
[27]	train-error:0.067528	evaluate-error:0.070849
[28]	train-error:0.067482	evaluate-error:0.070664
[29]	train-error:0.067482	evaluate-error:0.070295
[30]	train-error:0.067251	evaluate-error:0.07048
[31]	train-error:0.067066	evaluate-error:0.069926
[32]	train-error:0.067159	evaluate-error:0.069557
[33]	train-error:0.066882	evaluate-error:0.069004
[34]	train-error:0.06679	evaluate-error:0.068635
[35]	train-error:0.066882	evaluate-error:0.068635
[36]	tr

Multiple eval metrics have been passed: 'evaluate-error' will be used for early stopping.

Will train until evaluate-error hasn't improved in 25 rounds.
[1]	train-error:0.070756	evaluate-error:0.078044
[2]	train-error:0.069926	evaluate-error:0.076753
[3]	train-error:0.069926	evaluate-error:0.076753
[4]	train-error:0.070203	evaluate-error:0.076753
[5]	train-error:0.069834	evaluate-error:0.076937
[6]	train-error:0.069972	evaluate-error:0.076937
[7]	train-error:0.070572	evaluate-error:0.078413
[8]	train-error:0.070618	evaluate-error:0.078413
[9]	train-error:0.070157	evaluate-error:0.078413
[10]	train-error:0.070157	evaluate-error:0.078413
[11]	train-error:0.069649	evaluate-error:0.077675
[12]	train-error:0.06928	evaluate-error:0.077306
[13]	train-error:0.069234	evaluate-error:0.077306
[14]	train-error:0.069188	evaluate-error:0.077122
[15]	train-error:0.069142	evaluate-error:0.076568
[16]	train-error:0.068773	evaluate-error:0.076753
[17]	train-error:0.068358	evaluate-error:0.076015
[18]	tr